In [18]:
import pandas as pd

df = pd.read_csv(r"C:\Users\User\Desktop\Machine Learning\ML_Task_01\dataset_combined.csv")
print(df)

                                                  Path     Identity  \
0    C:\Users\User\Desktop\Machine Learning\Persona...  Hamza Ahmad   
1    C:\Users\User\Desktop\Machine Learning\Persona...  Hamza Ahmad   
2    C:\Users\User\Desktop\Machine Learning\Persona...  Hamza Ahmad   
3    C:\Users\User\Desktop\Machine Learning\Persona...  Hamza Ahmad   
4    C:\Users\User\Desktop\Machine Learning\Persona...  Hamza Ahmad   
..                                                 ...          ...   
194  C:\Users\User\Desktop\Machine Learning\Other\G...      Unknown   
195  C:\Users\User\Desktop\Machine Learning\Other\G...      Unknown   
196  C:\Users\User\Desktop\Machine Learning\Other\G...      Unknown   
197  C:\Users\User\Desktop\Machine Learning\Other\G...      Unknown   
198  C:\Users\User\Desktop\Machine Learning\Other\G...      Unknown   

      Expression  Age  
0        Smiling   23  
1    Not Smiling   23  
2    Not Smiling   23  
3        Smiling   23  
4    Not Smiling   23  
.. 

In [19]:
import numpy as np
from sklearn.model_selection import train_test_split


train, test = train_test_split(df, test_size=0.3, random_state = 8)

In [20]:
train.loc[train['Age'] == 23].describe()

,Age
count,73.0
mean,23.0
std,0.0
min,23.0
25%,23.0
50%,23.0
75%,23.0
max,23.0


In [21]:
train.loc[train['Age'] != 23].describe() 
#stratifed split is successful

,Age
count,66.000000
mean,58.136364
std,17.951976
min,12.000000
25%,46.000000
50%,59.500000
75%,72.000000
max,99.000000


In [22]:
X_data = train['Path']
X_tdata = test['Path']
y1_train = train['Identity']
y2_train = train['Expression']
y3_train = train['Age']
y1_test = test['Identity']
y2_test = test['Expression']
y3_test = test['Age']

In [23]:
X_data

182    C:\Users\User\Desktop\Machine Learning\Other\F...
113    C:\Users\User\Desktop\Machine Learning\Other\A...
79     C:\Users\User\Desktop\Machine Learning\Persona...
142    C:\Users\User\Desktop\Machine Learning\Other\C...
188    C:\Users\User\Desktop\Machine Learning\Other\G...
                             ...                        
136    C:\Users\User\Desktop\Machine Learning\Other\C...
133    C:\Users\User\Desktop\Machine Learning\Other\B...
105    C:\Users\User\Desktop\Machine Learning\Other\A...
84     C:\Users\User\Desktop\Machine Learning\Persona...
195    C:\Users\User\Desktop\Machine Learning\Other\G...
Name: Path, Length: 139, dtype: object

In [24]:
import skimage
from skimage import io
from skimage import transform

lst = []
for path in X_data:
    #print(path)
    curr_image = io.imread(path)
    img_resize = transform.resize(curr_image, output_shape=(32,32,3)) #resize image to match a certain type
    #img_shape = np.shape(img_resize)
    #print(img_shape)
    img_reshape = np.reshape(img_resize, (32*32*3))
    lst.append(img_reshape/255.)
    
X_train = np.array(lst)
print(X_train)

[[0.00020528 0.00021926 0.00024333 ... 0.00062626 0.00068319 0.00088945]
 [0.00118929 0.00121232 0.00106665 ... 0.00080002 0.00084119 0.00091485]
 [0.00223232 0.00152865 0.00110605 ... 0.00096045 0.00081889 0.00078292]
 ...
 [0.00133537 0.00099903 0.00115933 ... 0.00213558 0.00232012 0.00253542]
 [0.0006189  0.00088462 0.00140921 ... 0.00023611 0.00023125 0.00039626]
 [0.00024174 0.00024354 0.0002437  ... 0.00065935 0.00061324 0.00053635]]


In [25]:
X_train.ndim #Image dimensions after resizing

2

In [26]:
lst = []
for path in X_tdata:
    curr_image = io.imread(path)
    img_resize = transform.resize(curr_image, output_shape=(32,32,3))
    img_reshape = np.reshape(img_resize, (32*32*3))
    lst.append(img_reshape/255.)
    
X_test = np.array(lst)
print(X_test)

[[0.00372479 0.0036633  0.00364792 ... 0.00092841 0.00082061 0.00067896]
 [0.00369036 0.00335115 0.00285218 ... 0.0009648  0.00092806 0.00090443]
 [0.00148349 0.00128357 0.00114516 ... 0.00103969 0.0008479  0.00067647]
 ...
 [0.0006255  0.00047171 0.00045633 ... 0.00056502 0.00045768 0.00056406]
 [0.00270232 0.00256153 0.00237478 ... 0.00047237 0.00053545 0.00058865]
 [0.00389981 0.00390084 0.00388917 ... 0.00392059 0.00392001 0.0039204 ]]


In [27]:
X_test.shape

(60, 3072)

In [28]:
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import RidgeCV, ElasticNetCV, LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import joblib

#first we will define the hyperparameters we want to search over
params = {
    'penalty': ['l1', 'l2'],
    'alpha': [0.01, 0.1, 1.0],
    'max_iter': [1000, 5000],
    'tol': [1e-4, 1e-3],
    'eta0': [0.001, 0.01]
}

#initialize the linear regression model
model = SGDRegressor()

#Now create a GridSearchCV object and fit it to the data
grid_search = GridSearchCV(model, params, cv=5)
grid_search.fit(X_train, y3_train)

best_hyperparam = grid_search.best_params_

# print the best hyperparameters and the corresponding score
print("Best hyperparameters: ", best_hyperparam)
print("Best score: ", grid_search.best_score_)

#now saving the best model
best_model = grid_search.best_estimator_
joblib.dump(best_model, "SGD_Regressor.pkl")

# Print the learned regression coefficients
print("Regression Coefficients: ", best_model.coef_)

#making predictions 
y_pred = best_model.predict(X_test)

mse = mean_squared_error(y3_test, y_pred)
r2 = r2_score(y3_test, y_pred)
print("MSE:", mse)
print("R Squared Error:", r2)

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_mod

Best hyperparameters:  {'alpha': 0.01, 'eta0': 0.01, 'max_iter': 5000, 'penalty': 'l2', 'tol': 0.0001}
Best score:  -0.08381166626584416
Regression Coefficients:  [-0.85608256 -0.74739195 -0.68951357 ... -0.59753231 -0.61522556
 -0.72994991]
MSE: 517.5332288940272
R Squared Error: 0.0027893215824725814


In [29]:
#now 
# define the hyperparameters we want to search over for each model
ridge_params = {'alphas': [0.1, 1.0, 10.0], 'cv': 5}
elasticnet_params = {'l1_ratio': [0.1, 0.5, 0.9], 'alphas': [0.1, 1.0, 10.0], 'cv': 5}
lasso_params = {'alphas': [0.1, 1.0, 10.0], 'cv': 5}

# initialize the models with the defined hyperparameters
ridge_model = RidgeCV(**ridge_params)
elasticnet_model = ElasticNetCV(**elasticnet_params)
lasso_model = LassoCV(**lasso_params)

# fit each model to the training data
ridge_model.fit(X_train, y3_train)
elasticnet_model.fit(X_train, y3_train)
lasso_model.fit(X_train, y3_train)

# print the best hyperparameters and the corresponding score for each model
print("RidgeCV: Best hyperparameters: ", ridge_model.alpha_, " Best score: ", ridge_model.score(X_test, y3_test))
print("ElasticNetCV: Best hyperparameters: ", elasticnet_model.alpha_, elasticnet_model.l1_ratio_, " Best score: ", elasticnet_model.score(X_test, y3_test))
print("LassoCV: Best hyperparameters: ", lasso_model.alpha_, " Best score: ", lasso_model.score(X_test, y3_test))

#now saving the best model
best_model = elasticnet_model
joblib.dump(best_model, "ElasticNetCV.pkl")

# Print the learned regression coefficients
print("Regression Coefficients: ", best_model.coef_)

#making predictions 
y_pred = best_model.predict(X_test)

mse = mean_squared_error(y3_test, y_pred)
r2 = r2_score(y3_test, y_pred)
print("MSE:", mse)
print("R Squared Error:", r2)

RidgeCV: Best hyperparameters:  0.1  Best score:  0.21596772391941632
ElasticNetCV: Best hyperparameters:  0.1 0.1  Best score:  -0.05310589588005099
LassoCV: Best hyperparameters:  10.0  Best score:  -0.053444198746992466
Regression Coefficients:  [-0.0024168 -0.        -0.        ... -0.        -0.        -0.       ]
MSE: 546.5417754320755
R Squared Error: -0.05310589588005099


In [30]:
#model.score(X_train, y3_train) #performing slightly better on training data 